In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os

In [2]:
%pwd

'C:\\Users\\Saurav Kumar\\PycharmProjects\\summer2025\\MLP_with_MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\Saurav Kumar\\PycharmProjects\\summer2025\\MLP_with_MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.mlProject.constants import *

In [7]:
from src.mlProject.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url, #changed from URL to url for both sides of =
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [23]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url, #changed to lowercase url instead of URL
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-19 15:47:24,063: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-19 15:47:24,066: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-19 15:47:24,068: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-19 15:47:24,070: INFO: common: created directory at: artifacts]
[2025-07-19 15:47:24,072: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-19 15:47:24,903: INFO: 1989158804: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 25115
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "ad338b467d99f03caeaf8d776d3b2e0d172221f9f138f0097afddcc4fc013e15"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D1E9:2CB420:1C677F:280E69:687C207B
Accept-Ranges: bytes
Date: Sat, 19